# Generate automated session reports for all participants

In [ ]:
import pandas as pd
import numpy as np
import mne
import mne_bids
import glob
import warnings
import os
import re
import subprocess

import q1k_pyll_tools as qpt

warnings.filterwarnings('ignore')

### Select task parameters and file paths

In [ ]:
# Select kind of data experimental or control group

#dataset_group = "experimental"

#if dataset_group == "control":
#    # Control group data
#    project_path = "/home/james/q1k/pilot/q1k-external-pilot/"
#    task_id_in = "ap"
#    task_id_in_et = "ap"
#    task_id_out = "ap"
#    #subject_id = '002'
#    session_id = '01'
#    run_id = '1'

#elif dataset_group == "experimental":
## Experimental group data

project_path = "/project/def-emayada/q1k/experimental/"
pyll_path = "derivatives/pylossless/"
code_path = "code/q1k_eeget_init/"
task_id = "RS"
#task_id_in_et = "TO"
#task_id_out = "TO"
#subject_id = 'Q1K_HSJ_100123_F1'
run_id = '1'
session_id = '01'
root_data_path = "" 


In [ ]:
#find existing *.out files 
slurm_out_path = project_path + pyll_path + code_path + 'slurm_output/' +task_id + '/'
slurm_out_files = slurm_out_path + '*' + task_id + '*.out'
processed_sessions = glob.glob(slurm_out_files, recursive=True)
processed_ids = [re.search(r"sub-(.+?)_ses", path).group(1) for path in processed_sessions]

pattern = project_path + 'sub-*/**/eeg/*' + task_id + '*.edf'
file_paths = glob.glob(pattern, recursive=True)
for file_path in file_paths:
    file_name = os.path.basename(file_path)  # Extract the filename from the full path
    print(f"File Path: {file_path}, File Name: {file_name}")
    file_id = re.search(r"sub-(.+?)_ses", file_name).group(1)
    print(f"File ID: {file_id}")
    
    # Skip sessions that have already been processed
    if file_id in processed_ids:
        print(file_id + ' has already been processed')
        continue

    sbatch_command, result = qpt.submit_slurm_job(project_path, file_name, task_id)